In [1]:
import glob
import pandas as pd
import json
import datetime
from tqdm import tqdm
import os
from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm, tqdm_notebook
import glob
import numpy as np

from django.db.models import Min,Max,Count
from sklearn.neighbors import KDTree
comfig_file_path = '../../main_config.txt'
main_name = 'yandex/'
sub_name = 'raw_org_search/'
import time
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

/home/tserekh/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/home/tserekh/env/lib/python3.6/site-packages/django/db/models/base.py:299: RuntimeWarning: Model 'main_map.predictedsquare' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)


In [2]:
import sys
import json
sys.path.append('/usr/lib/python3/dist-packages')
from main_map.models import OrganizationNatClass, House, Metro, DistanceTemp, City

In [3]:
#nat_classes = ['АЗС','Парикмахерская', 'Ресторан','Супермаркет']
true_nat_classes = ['Супермаркет']
#nat_classes = ['АЗС']
#nat_classes = ['Автомойка']

In [4]:
lat_min, lon_min = 54.305605, 33.864050
lat_max, lon_max = 57.518175, 41.422644




filter_geo =    {
    'lat__gt' : lat_min, 
    'lon__gt' : lon_min,
    'lat__lt' : lat_max, 
    'lon__lt' : lon_max,
}
filter_dic = filter_geo.copy()

In [5]:
columns = ['address', 'chain_name', 'nat_class', 'source_id','source_name', 'x', 'y']

filter_dic['nat_class__in'] = true_nat_classes
df_orgs1 = pd.DataFrame(list(OrganizationNatClass.objects.filter(**filter_dic).values().values_list(*columns)),columns=columns)
df_orgs1 = df_orgs1.rename(columns = {'source_id':'source_id1','source_name':'source_name1'})
df_orgs1['true'] = True

In [6]:
df_orgs2 = pd.DataFrame(
    list(OrganizationNatClass
         .objects.filter(**filter_geo)
         .exclude(**{'source_id__in':df_orgs1['source_id1'].values})
         .values()
         .values_list(*columns)[:10000]),columns=columns)

In [7]:
df_orgs2 = df_orgs2.rename(columns = {'source_id':'source_id1','source_name':'source_name1'})
df_orgs2['true'] = False

In [8]:
df_orgs = pd.concat([df_orgs1, df_orgs2])
#del df_orgs1, df_orgs2

In [9]:
columns = ['address', 'flat_num', 'source_id', 'source_name',  'x', 'y']
df_homes = pd.DataFrame(list(House.objects.filter(**filter_geo).values().values_list(*columns)),columns=columns)

In [10]:
xy = ['x','y']

In [11]:
tree = KDTree(df_orgs[xy])
k = 40

distances_list, indexes_list = tree.query(df_homes[xy], k=k)
df_orgs_all = pd.DataFrame()
for i in tqdm_notebook(range(k)):
    indexes = indexes_list[:,i]
    distances = distances_list[:,i]
    df_orgs_sub = df_orgs.iloc[indexes].copy()
    
#     df_orgs_sub['source_id2'] = df_orgs.iloc[indexes].copy()['source_id'].values
#     df_orgs_sub['source_name2'] = df_orgs.iloc[indexes].copy()['source_name'].values
    
    df_orgs_sub = df_orgs_sub.rename(columns = {'source_id':'source_id1','source_name':'source_name1'})

    df_orgs_sub['flat_num'] = df_homes['flat_num'].values
    df_orgs_sub['distance'] = distances
    df_orgs_all = df_orgs_all.append(df_orgs_sub)

In [12]:
len(indexes)

94525

In [13]:
len(df_homes)

94525

In [14]:
def get_weight(distance, step_size, slope_size):
    score =  (distance - step_size)/slope_size
    if score>1:
        score = 1
    elif score<0:
        score = 0
    return 1 - score

In [15]:
step_size = 50
slope_sizes = [100,200,300,400,500,1000]
slope_size = 500

In [16]:
from sklearn.metrics import roc_auc_score

In [17]:
def f(step_size, slope_size):
    spec_get_weight = lambda x:get_weight(x, step_size, slope_size)
    df_orgs_all['score'] = df_orgs_all['distance'].apply(spec_get_weight)
    
    df_gb1 = df_orgs_all[df_orgs_all['true']].groupby(['source_id1','source_name1'],as_index=False)['score'].sum()
    df_gb2 = df_orgs_all[df_orgs_all['true']==False].groupby(['source_id1','source_name1'],as_index=False)['score'].sum()
    df_gb = pd.concat([df_gb1, df_gb2])

    df = df_orgs_all.copy()
    df = pd.merge(df, df_gb,on=['source_id1','source_name1'])
    df['rate'] = df['score_x']/(df['score_y'] + df['score_x']*(df['true']==False))
    df['our_flats'] = df['flat_num']*df['rate'] 

    agg = {}
    agg['our_flats'] = sum
    agg['true'] = lambda x: list(x)[0]
    df_result = df.groupby(['source_id1','source_name1'],as_index=False).agg(agg)
    
    
    auc_roc = roc_auc_score(df_result['true'], df_result['our_flats'])
    return auc_roc

In [18]:
from bayes_opt import BayesianOptimization

In [19]:
opt = BayesianOptimization(f,{'step_size': (0, 100), 'slope_size': (0, 1000)})

In [20]:
opt.maximize()

Initialization
----------------------------------------------------------
 Step |   Time |      Value |   slope_size |   step_size | 
    1 | 00m10s |    0.59016 |     466.1499 |     20.3734 | 
    2 | 00m10s |    0.53280 |     675.4614 |     47.4773 | 
    3 | 00m09s |    0.59325 |     464.7085 |      9.4418 | 
    4 | 00m10s |    0.52713 |     697.8831 |     56.1194 | 
    5 | 00m10s |    0.51016 |     817.3920 |     45.8262 | 


/home/tserekh/env/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
----------------------------------------------------------
 Step |   Time |      Value |   slope_size |   step_size | 
    6 | 00m16s |    0.44621 |       0.9399 |      1.6406 | 
    7 | 00m13s |    0.66063 |     292.8441 |      0.0992 | 
    8 | 00m12s |    0.65454 |     209.3009 |     99.9017 | 
    9 | 00m17s |    0.61911 |     287.9955 |     99.7804 | 


KeyboardInterrupt: 